# hicstuff API demo

All steps of hicstuff pipeline and downstream operations can be ran using the python api. The matrix can be generated directly from the reads, just like in the command line or by doing each step separately for fine control.


## Data preparation
If using minimap2, the genome can be in fasta format. If using bowtie2, it must be indexed using bowtie2-build:
```bash
bowtie2-build genome.fasta genome
```
The input reads can be in fastq format, or if already aligned to the genome in SAM/BAM format. The pipeline also accepts input in the form of pairs file. The input format is specified using the `start_stage` argument (fastq/sam/pairs/pairs_idx).

## Full pipeline using the API
The hicstuff.pipeline submodule allows to run all steps at one, in a way identical to the hicstuff pipeline command.

In [ ]:
import hicstuff.pipeline as hpi

hpi.full_pipeline(input1='forward.fq', input2='reverse.fq', genome='genome', enzyme="DpnII")